In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/PyAI/13 word_Embedding

Mounted at /content/drive
/content/drive/MyDrive/PyAI/13 word_Embedding


In [2]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""),".."))

import custom
import numpy
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
with open("ptb.train.txt") as f:
    text = f.readlines()
word_dict, number_dict = custom.make_dict(text)
print(len(word_dict))

10000


In [4]:
corpus = custom.word_num_encoding(text, word_dict)
comatrix = custom.make_comatrix(corpus, len(word_dict), window_size=2)
print(comatrix)
print(comatrix.shape)

[[0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 5.0844e+04 0.0000e+00 ... 0.0000e+00 1.0000e+00 2.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 1.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 2.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]]
(10000, 10000)


In [5]:
word_pair = custom.make_word_pair(comatrix)
print(word_pair)
print(word_pair.shape)

[[   1    1]
 [   1    9]
 [   1   10]
 ...
 [9999 4190]
 [9999 6191]
 [9999 9506]]
(874435, 2)


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(word_pair[:,0], dtype = torch.long, device = device)
tensor_t = torch.tensor(word_pair[:,1], dtype = torch.long, device = device)
zip_list = list(zip(tensor_x, tensor_t))
dataloader = DataLoader(zip_list,batch_size=1000,shuffle=True)

tensor_x = None
tensor_t = None
zip_list = None

In [7]:


class NN(nn.Module) :
    def __init__(self, word_size) :
        super().__init__();
        self.f = nn.Embedding(word_size, 100, max_norm=1, padding_idx=0)
        self.g = nn.Linear(100, word_size)
    def forward(self, x) :
        y = self.f(x)
        y = self.g(y)
        return y

F = NN(len(word_dict))
F = F.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(F.parameters(), lr =0.001)
epoch = 100

for e in range(epoch) :
    loss_sum = 0
    for x, t in dataloader :
        y = F(x)

        loss = loss_function(y, t)
        loss_sum += loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    loss_sum /= len(dataloader)

    if (e+1) % 1 == 0 :
        print("epoch {} | loss {}".format(e+1, loss_sum))


epoch 1 | loss 8.625601768493652
epoch 2 | loss 8.02645492553711
epoch 3 | loss 7.9782562255859375
epoch 4 | loss 7.942793369293213
epoch 5 | loss 7.902358531951904
epoch 6 | loss 7.860827922821045
epoch 7 | loss 7.819523334503174
epoch 8 | loss 7.779201507568359
epoch 9 | loss 7.740523815155029
epoch 10 | loss 7.703644275665283
epoch 11 | loss 7.668661594390869
epoch 12 | loss 7.635254383087158
epoch 13 | loss 7.603918075561523
epoch 14 | loss 7.574310779571533
epoch 15 | loss 7.545964241027832
epoch 16 | loss 7.519214630126953
epoch 17 | loss 7.493711948394775
epoch 18 | loss 7.469409942626953
epoch 19 | loss 7.446224689483643
epoch 20 | loss 7.424317359924316
epoch 21 | loss 7.403304100036621
epoch 22 | loss 7.383484840393066
epoch 23 | loss 7.364320755004883
epoch 24 | loss 7.345884323120117
epoch 25 | loss 7.3284406661987305
epoch 26 | loss 7.3117170333862305
epoch 27 | loss 7.295662879943848
epoch 28 | loss 7.280313014984131
epoch 29 | loss 7.2655792236328125
epoch 30 | loss 7.25

In [8]:
F.to("cpu")
word_vecs = F.state_dict()['f.weight'].numpy()

print(word_vecs.shape)
print(word_vecs)

(10000, 100)
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.03206774  0.05362986  0.0394065  ... -0.05453866 -0.05378624
   0.07099456]
 [-0.08107498  0.14124154  0.12616363 ... -0.09763073 -0.09737623
  -0.14953932]
 ...
 [ 0.05133586 -0.0147954   0.04445554 ... -0.12503707 -0.0347121
  -0.11146565]
 [ 0.07802434 -0.02097864  0.02581932 ...  0.01747462  0.11716229
  -0.1016208 ]
 [ 0.13433488 -0.09984097  0.03477243 ...  0.16088031 -0.03811574
  -0.09786693]]


In [9]:
word_vecs_list = word_vecs.tolist()
vector_dict = {}
words = list(word_dict.keys())

for i in range(len(word_vecs_list)) :
    vector_dict[words[i]] = word_vecs_list[i]

print(len(vector_dict))

10000


In [11]:
querys = ["you", "year", "car", "toyota"]
for q in querys :
    result = custom.most_similiar(q, word_dict, number_dict, word_vecs, top = 5, mode = "euc")
    print(f"검색어 || {q}")
    for w, s in result :
        print(f"{w} : {s}")
    print()

검색어 || you
<pad> : 0.9934328198432922
it : 1.0217151641845703
somebody : 1.0229167938232422
i : 1.0267128944396973
can : 1.0331823825836182

검색어 || year
this : 0.8241734504699707
last : 0.8276179432868958
<pad> : 0.9533320665359497
<unk> : 0.960233747959137
to : 0.9604543447494507

검색어 || car
<pad> : 1.00050687789917
luxury : 1.0649042129516602
cars : 1.0792003870010376
factory : 1.1049634218215942
<unk> : 1.1087225675582886

검색어 || toyota
<pad> : 1.0156124830245972
honda : 1.0653148889541626
infiniti : 1.0772230625152588
morale : 1.07987380027771
lexus : 1.0878597497940063



In [12]:
querys = ["you", "year", "car", "toyota"]
for q in querys :
    result = custom.most_similiar(q, word_dict, number_dict, word_vecs, top = 5, mode = "cos")
    print(f"검색어 : {q}")
    for w, s in result :
        print(f"{w} : {s}")
    print()

검색어 : you
somebody : 0.476644366979599
i : 0.46429523825645447
can : 0.44903555512428284
we : 0.41516685485839844
your : 0.4145917296409607

검색어 : year
last : 0.6361261606216431
this : 0.6193463206291199
next : 0.4601631462574005
week : 0.4532422721385956
earlier : 0.44665607810020447

검색어 : car
luxury : 0.4323689937591553
cars : 0.4185750186443329
factory : 0.39209216833114624
high-interest : 0.3815464377403259
auto : 0.3782140612602234

검색어 : toyota
honda : 0.44585326313972473
infiniti : 0.4330969452857971
morale : 0.4244190454483032
eli : 0.41949954628944397
lexus : 0.41837313771247864



In [13]:
with open("word2vec_vector.pkl", mode = "wb") as f :
    pickle.dump(word_vecs, f)